In [33]:
import psycopg2
import csv

conn = psycopg2.connect("host=localhost port=5432 dbname=odscourse user=postgres password=asecurepassword")
cursor = conn.cursor()
import pandas as pd
import numpy as np
df = pd.read_csv('athlete_events.csv')
# Clear data, change NaN to 0
df = df.fillna(0)
df['Age'] = df['Age'].astype("Int64")
df['Height'] = df['Height'].astype("Int64")

df.to_csv('for_sql.csv')


In [3]:
query = """
CREATE TABLE IF NOT EXISTS olimpic(
main_id SERIAL PRIMARY KEY,
id INTEGER,
Name VARCHAR,
Sex VARCHAR,
Age INTEGER,
Height INTEGER,
Weight FLOAT,
Team VARCHAR,
NOC VARCHAR,
Games VARCHAR,
Year INTEGER,
Season VARCHAR,
City VARCHAR,
Sport VARCHAR,
Event VARCHAR,
Medal VARCHAR)"""


In [7]:
cursor.execute(query)
conn.commit()

In [ ]:
with open('for_sql.csv', 'r') as f:
    reader = csv.reader(f)
    # Skip the header row
    next(reader)
    for ID, row in enumerate(reader):
        cursor.execute(
            "INSERT INTO olimpic VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)",
            row
        )
conn.commit()

In [8]:
from tabulate import tabulate
from IPython.display import HTML, display

# Function to run queries
def fetch_all(cursor):
    colnames = [desc[0] for desc in cursor.description]
    records = cursor.fetchall()
    print(display(HTML(tabulate([{colname:value for colname, value in zip(colnames, record)} for record in records], tablefmt='html')))) 


cursor.execute(
    """
    SELECT * FROM olimpic Limit 5
    
    """
)
fetch_all(cursor)

0,1,A Dijiang,M,24,180,80,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,0
1,2,A Lamusi,M,23,170,60,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,0
2,3,Gunnar Nielsen Aaby,M,24,0,0,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,0
3,4,Edgar Lindenau Aabye,M,34,0,0,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21,185,82,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,0


None


__1. How old were the youngest male and female participants of the 1996 Olympics?__

- 16 and 15
- **14 and 12**
- 16 and 12
- 13 and 11

In [9]:
#Find the youngest Male
cursor.execute(
    """
    SELECT MIN(Age) FROM olimpic WHERE Sex = 'M' and Age > 0 and Year = 1996
    
    """
)
fetch_all(cursor)

14


None


In [10]:
#Find the youngest Female
cursor.execute(
    """
    SELECT MIN(Age) FROM olimpic WHERE Sex = 'F' and Age > 0 and Year = 1996
    
    """
)
fetch_all(cursor)

12


None


__2. What was the percentage of male gymnasts among all the male participants of the 2000 Olympics? Round the answer to the first decimal.__

*Hint:* here and further if needed drop duplicated sportsmen to count only unique ones. 

- 0.2
- **1.5** 
- 2.5
- 7.7

In [ ]:
# Create Tables to keep data about all gymnasts (gall) and all Male gymnasts (gm)
cursor.execute(
    """
CREATE TABLE gall(Name Varchar);
  CREATE TABLE gm(Name Varchar);
    
    
    """
)
conn.commit()
fetch_all(cursor)

In [289]:
# Fill these tables (postfresSQL does not allow to work with CTE, so we have to create tables in order to get rid of repeatitions)
cursor.execute(
    """
   WITH t_deleted AS
(SELECT * FROM olimpic WHERE Year = 2000 and Sex = 'M'), -- 1st step
t_inserted AS
(SELECT *, ROW_NUMBER() OVER(PARTITION BY id ORDER BY name) rnk
    FROM t_deleted ) -- 2nd step
INSERT INTO gall SELECT Name FROM t_inserted 
WHERE rnk=1;
  WITH t_deleted AS
(SELECT * FROM olimpic WHERE Sex = 'M' and Year = 2000 and Sport = 'Gymnastics'), -- 1st step
t_inserted AS
(SELECT *, ROW_NUMBER() OVER(PARTITION BY id ORDER BY name) rnk
    FROM t_deleted ) -- 2nd step
INSERT INTO gm SELECT Name FROM t_inserted 

WHERE rnk=1;


     """
)
conn.commit()

In [295]:
# Calculate the percentage
cursor.execute(
    """
 SELECT
  (SELECT Count(gm.Name) from gm )/ (SELECT COUNT(gall.Name) FROM gall) ::numeric * 100 as result


     """
)
fetch_all(cursor)

1.47439


None


In [298]:
# get rid of temporary tables
cursor.execute(
    """
DROP TABLE gall;
DROP TABLE gm;
    
    
    """
)
conn.commit()
#fetch_all(cursor)


__3. What are the mean and standard deviation of height for female basketball players participated in the 2000 Olympics? Round the answer to the first decimal.__

- 178.5 and 7.2
- 179.4 and 10
- 180.7 and 6.7
- **182.4 and 9.1**

In [297]:
conn.commit()

In [306]:
#Create table to keep all the data without repeatitions by name
query = """
CREATE TABLE IF NOT EXISTS olimpic_new(
main_id SERIAL PRIMARY KEY,
id INTEGER,
Name VARCHAR,
Sex VARCHAR,
Age INTEGER,
Height INTEGER,
Weight FLOAT,
Team VARCHAR,
NOC VARCHAR,
Games VARCHAR,
Year INTEGER,
Season VARCHAR,
City VARCHAR,
Sport VARCHAR,
Event VARCHAR,
Medal VARCHAR);
CREATE TABLE tmp (
main_id SERIAL PRIMARY KEY,
id INTEGER,
Name VARCHAR,
Sex VARCHAR,
Age INTEGER,
Height INTEGER,
Weight FLOAT,
Team VARCHAR,
NOC VARCHAR,
Games VARCHAR,
Year INTEGER,
Season VARCHAR,
City VARCHAR,
Sport VARCHAR,
Event VARCHAR,
Medal VARCHAR,
rnk INTEGER)
"""
cursor.execute(query)
conn.commit()

In [312]:
# fill this table
cursor.execute(
    """
    
   WITH t_deleted AS
(SELECT * FROM olimpic), -- 1st step
t_inserted AS
(SELECT *, ROW_NUMBER() OVER(PARTITION BY id ORDER BY id) rnk
    FROM t_deleted) -- 2nd step
INSERT INTO tmp SELECT * FROM t_inserted 
WHERE rnk=1; -- 3rd step 

ALTER TABLE tmp DROP COLUMN rnk;

INSERT INTO olimpic_new SELECT * FROM tmp;

DROP TABLE tmp; 
     """
)
conn.commit()

In [359]:
# calculate the mean and std using standart functions
cursor.execute(
 """
 
 SELECT   avg(olimpic_new.Height), stddev(olimpic_new.Height) FROM olimpic_new
     WHERE Sex = 'F' and Sport = 'Basketball' and Year = 2000 and Height > 0
 """
)
fetch_all(cursor)


182.5,9.13241


None


In [358]:
conn.commit()

__4. Find a sportsperson participated in the 2002 Olympics, with the highest weight among other participants of the same Olympics. What sport did he or she do?__

- Judo
- **Bobsleigh**
- Weightlifting
- Boxing

In [372]:
# Get the heaviest sportperson
cursor.execute(
 """
 
 
 SELECT Sport FROM olimpic_new
     WHERE Year = 2002 and  Weight = (SELECT MAX(Weight) FROM olimpic_new WHERE Year = 2002) Limit 1
 """
)
fetch_all(cursor)


Bobsleigh


None


In [370]:
conn.commit()

__5. How many times did Pawe Abratkiewicz participate in the Olympics held in different years?__

- 0
- 1 
- 2
- ***3*** 

In [412]:
#create a temporary table in order to keep there all the sportspersons not repeating pair of Name and Year of participation
cursor.execute(
 """
  CREATE TABLE tmp(Name VARCHAR, Year VARCHAR);  
   WITH t_deleted AS
(SELECT * FROM olimpic), -- 1st step
t_inserted AS
(SELECT *, ROW_NUMBER() OVER(Partition BY Name, Year ORDER BY Year) rnk
    FROM t_deleted) -- 2nd step
INSERT INTO tmp SELECT Name, Year FROM t_inserted 
WHERE rnk=1; -- 3rd step 


 """
)

conn.commit()
cursor.execute(
"""
SELECT * FROM olimpic
     WHERE Name = 'Pawe Abratkiewicz';

""")
fetch_all(cursor)


997,562,Pawe Abratkiewicz,M,21,183,84,Poland,POL,1992 Winter,1992,Winter,Albertville,Speed Skating,Speed Skating Men's 500 metres,0
998,562,Pawe Abratkiewicz,M,21,183,84,Poland,POL,1992 Winter,1992,Winter,Albertville,Speed Skating,"Speed Skating Men's 1,000 metres",0
999,562,Pawe Abratkiewicz,M,27,183,84,Poland,POL,1998 Winter,1998,Winter,Nagano,Speed Skating,Speed Skating Men's 500 metres,0
1000,562,Pawe Abratkiewicz,M,27,183,84,Poland,POL,1998 Winter,1998,Winter,Nagano,Speed Skating,"Speed Skating Men's 1,000 metres",0
1001,562,Pawe Abratkiewicz,M,31,183,84,Poland,POL,2002 Winter,2002,Winter,Salt Lake City,Speed Skating,Speed Skating Men's 500 metres,0
1002,562,Pawe Abratkiewicz,M,31,183,84,Poland,POL,2002 Winter,2002,Winter,Salt Lake City,Speed Skating,"Speed Skating Men's 1,000 metres",0


None


In [413]:
cursor.execute(
"""
SELECT Count(*) FROM olimpic
     WHERE Name = 'Pawe Abratkiewicz';

""")
fetch_all(cursor)

6


None


In [414]:
# get rid of the temporary table
cursor.execute("DROP TABLE tmp; ")
conn.commit()

__6. How many silver medals in tennis did Australia win at the 2000 Olympics?__

- 0
- 1 
- **2** 
- 3 

In [419]:
# find out all the Silver medals won by Australia
cursor.execute(
"""
SELECT Count(*) FROM olimpic
     WHERE Medal = 'Silver' and Sport = 'Tennis' and Year = 2000 and Team = 'Australia';

""")
fetch_all(cursor)

2


None


In [418]:
conn.commit()

__7. Is it true that Switzerland won fewer medals than Serbia at the 2016 Olympics? Do not consider NaN values in _Medal_ column.__ 

- **Yes** 
- No

In [424]:
# Check the statement, here '0' represents NULL value (it cannot be NULL, because Weight is a Float and it does not work with NULLs)
cursor.execute(
"""
SELECT(SELECT Count(*) FROM olimpic_new
     WHERE Medal <> '0' and  Year = 2016 and Team = 'Serbia')
     > (SELECT COUNT(*) FROM olimpic_new
     WHERE Medal <> '0' and  Year = 2016 and Team = 'Switzerland');

""")
fetch_all(cursor)

True


None


In [423]:
conn.commit()

__8. What age category did the fewest and the most participants of the 2014 Olympics belong to?__
- [45-55] and [25-35) correspondingly
- **[45-55] and [15-25) correspondingly**
- [35-45] and [25-35) correspondingly
- [45-55] and [35-45) correspondingly

In [54]:
# create a table to keep all the age brackets and their number
cursor.execute(
"""
CREATE TABLE years(id INTEGER, cnt INTEGER);
INSERT INTO years VALUES(1, (SELECT COUNT(*) FROM olimpic_new WHERE Age >= 45 and Age <= 55));
INSERT INTO years VALUES(2, (SELECT COUNT(*) FROM olimpic_new WHERE Age >= 25 and Age < 35));
INSERT INTO years VALUES(3, (SELECT COUNT(*) FROM olimpic_new WHERE Age >= 35 and Age <= 45));
INSERT INTO years VALUES(4, (SELECT COUNT(*) FROM olimpic_new WHERE Age >= 35 and Age < 45));
INSERT INTO years VALUES(5, (SELECT COUNT(*) FROM olimpic_new WHERE Age >= 15 and Age < 25));
""")
conn.commit()
cursor.execute(
"""
Select * FROM years
""")
fetch_all(cursor)

1,1453
2,54783
3,6712
4,6468
5,65727


None


In [55]:
# Find id of the most populated group
cursor.execute(
"""
Select id FROM years WHERE cnt = (Select MAX(cnt) FROM years)

""")
fetch_all(cursor)

5


None


In [57]:
# Find id of the less populated group
cursor.execute(
"""
Select id FROM years WHERE cnt = (Select MIN(cnt) FROM years)

""")
fetch_all(cursor)

1


None


__9. Is it true that there were Summer Olympics held in Lake Placid? Is it true that there were Winter Olympics held in Sankt Moritz?__

- Yes, Yes
- Yes, No
- **No, Yes** 
- No, No 

In [425]:
# Simply check both statements
cursor.execute(
"""
SELECT(SELECT Count(*) > 0 FROM olimpic_new
     WHERE City = 'Lake Placid' and  Season = 'Summer') as first,
     (SELECT COUNT(*) > 0 FROM olimpic_new
     WHERE City = 'Sankt Moritz' and  Season = 'Winter') as second;

""")
fetch_all(cursor)

False,True


None


__10. What is the absolute difference between the number of unique sports at the 1995 Olympics and 2016 Olympics?__

- 16
- 24
- 26
- **34**

In [48]:
# create tables to keep All the sports from 1995 and 2016  games (PS there were no games in 1995)
cursor.execute(
 """
  CREATE TABLE s1995(Sport VARCHAR);  
   WITH t_deleted AS
(SELECT * FROM olimpic WHERE Year = 1995), -- 1st step
t_inserted AS
(SELECT *, ROW_NUMBER() OVER(Partition BY Sport ORDER BY Sport) rnk
    FROM t_deleted) -- 2nd step
INSERT INTO s1995 SELECT Sport FROM t_inserted 
WHERE rnk=1; -- 3rd step 

  CREATE TABLE s2016(Sport VARCHAR);  
   WITH t_deleted AS
(SELECT * FROM olimpic WHERE Year = 2016), -- 1st step
t_inserted AS
(SELECT *, ROW_NUMBER() OVER(Partition BY Sport ORDER BY Year) rnk
    FROM t_deleted) -- 2nd step
INSERT INTO s2016 SELECT Sport FROM t_inserted 
WHERE rnk=1; -- 3rd step 

 """
)

conn.commit()
cursor.execute(
"""
SELECT abs( (Select Count(*) from s1995) - (Select Count(*) from s2016))

""")
fetch_all(cursor)


34


None


In [47]:
#Get rid of tmp tables
cursor.execute(
 """
  DROP TABLE s1995;
  DROP TABLE s2016;

 """
)

conn.commit()

In [46]:

conn.commit()